#### Enter desired Lakehouse Name and Workspace Name to Optimize

In [1]:
# UsersProperty
workspace_name = 'FeatureOnyxToolsII'
lakehouse_name = "TEST00005"

print(f"Lakehouse to mount: {lakehouse_name}")
print(f"Workspace : {workspace_name}")

StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 3, Finished, Available, Finished)

Lakehouse to mount: TEST00005
Workspace : FeatureOnyxToolsII


In [2]:
import notebookutils
from notebookutils import mssparkutils
from pyspark.sql import SparkSession

StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 4, Finished, Available, Finished)

In [3]:
%run Fabric_utils

StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 5, Finished, Available, Finished)

In [4]:
workspace_id = get_workspace_id(workspace_name)
lakehouse_id = get_lakehouse_id(workspace_name, lakehouse_name)


StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 6, Finished, Available, Finished)

In [5]:

storage_url = f"abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}"
mount_point = f"/{lakehouse_name}"  

notebookutils.fs.mount(storage_url, mount_point)

mount_info = next((m for m in notebookutils.fs.mounts() if m.mountPoint == mount_point), None)

if mount_info:
    localPath = mount_info.localPath  
    print(f"localPath = {localPath}")

else:
    raise ValueError(f"Mount point {mount_point} not found!")


StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5d9d42af-5e0a-4474-8123-63e7cf96b3a6)

SynapseWidget(Synapse.DataFrame, 4467f400-8489-4a4c-9e6c-07f96e2cb86a)

localPath = /synfs/notebook/a1826132-a525-4178-b153-2b908779d6d3/TEST00005


#### TableSize Analysis

In [6]:
distinct_folder=mssparkutils.notebook.run("NB - TableSizeAnalysis",500,{
    "beforeAfter": "before",
    "lakehouse_name": lakehouse_name,
    "lakehouse_path": localPath
})

StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 8, Finished, Available, Finished)

In [ ]:
mssparkutils.notebook.run("NB - performance baseline",500,{
    "specificpath":f"{distinct_folder}/before",
    "lakehouse_name": lakehouse_name,
    "lakehouse_id" : lakehouse_id,
    "workspace_id" : workspace_id
})

#### Exclude shortcuts from maintenance

In [8]:
is_parent_call = True

StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 10, Finished, Available, Finished)

In [9]:
%run "NB - Shortcut Identification"

StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 13, Finished, Available, Finished)

In [10]:
shortcut_names = fetch_shortcut_names(workspace_id, lakehouse_id)
display(shortcut_names)

StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 14, Finished, Available, Finished)


Fetching Shortcuts...


SynapseWidget(Synapse.DataFrame, 0f1658b5-a41f-451a-a4bd-a4ab714fdeda)

#### Maintainance

In [11]:
from delta.tables import *
from datetime import datetime
import pandas as pd

spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

tables_base_path = f"file:{localPath}/Tables/"

tables_info = mssparkutils.fs.ls(tables_base_path)

table_names_df = pd.DataFrame([
    {
        "table_name": t.name.strip('/'),
        "table_path": tables_base_path + t.name.strip('/')
    }
    for t in tables_info
])

all_tables = table_names_df["table_name"].tolist()

shortcuts_list = shortcut_names["name"].tolist()

real_tables_df = table_names_df[~table_names_df["table_name"].isin(shortcuts_list)].reset_index(drop=True)

display(real_tables_df)

# 🧹 
def cleanTables(table_name, table_path):
    retention = 48  
    print(f"\n🔄 Optimizing and vacuuming: {table_name}")
    spark.sql(f"OPTIMIZE delta.`{table_path}` VORDER")
    spark.sql(f"VACUUM delta.`{table_path}` RETAIN {retention} HOURS")
    print(f"✅ Completed: {table_name}")

# 🔁 
for _, row in real_tables_df.iterrows():
    cleanTables(row["table_name"], row["table_path"])


StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 82e9bd56-5753-410a-9065-9e184b229e71)


🔄 Optimizing and vacuuming: dimension_customer
✅ Completed: dimension_customer

🔄 Optimizing and vacuuming: dimension_date
✅ Completed: dimension_date

🔄 Optimizing and vacuuming: dimension_employee
✅ Completed: dimension_employee

🔄 Optimizing and vacuuming: dimension_stock_item
✅ Completed: dimension_stock_item

🔄 Optimizing and vacuuming: fact_sale
✅ Completed: fact_sale


In [12]:
mssparkutils.notebook.run("NB - TableSizeAnalysis",500,{
     "beforeAfter": "after", 
     "distinct_folder": distinct_folder,
    "lakehouse_name": lakehouse_name,
    "lakehouse_path": localPath
})

StatementMeta(, a1826132-a525-4178-b153-2b908779d6d3, 16, Finished, Available, Finished)

'25fad05617b64cc69dcc3c49b35cbfd3'